In [2]:
from joblib import load

pca = load('pca.pkl')
clf = load('clf.pkl')


import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from skimage.feature import hog

def remove_noise(image):
    return cv2.GaussianBlur(image, (5, 5), 0)

def normalize_size(image, size=(64, 64)):
    return cv2.resize(image, size)

def adjust_contrast(image):
    img_yuv = cv2.cvtColor(image, cv2.COLOR_BGR2YUV)
    img_yuv[:, :, 0] = cv2.equalizeHist(img_yuv[:, :, 0])
    return cv2.cvtColor(img_yuv, cv2.COLOR_YUV2BGR)

def extract_hog_features(img):
    gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    features, hog_image = hog(gray_img, orientations=8, pixels_per_cell=(8, 8), cells_per_block=(2, 2), visualize=True)
    return np.array(features)

def extract_hog_features2(img):
    gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    features, hog_image = hog(gray_img, orientations=8, pixels_per_cell=(8, 8), cells_per_block=(2, 2), visualize=True)
    return hog_image

import numpy as np
from scipy import signal
import math
import matplotlib.pyplot as plt
import cv2
import time 
import pickle
import warnings
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import PCA
from skimage import feature

# Inicializar el clasificador de detección de rostros
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
warnings.filterwarnings("ignore")

device = 'cpu'
#
font                   = cv2.FONT_HERSHEY_SIMPLEX
bottomLeftCornerOfText = (10,30)
fontScale              = 1
fontColor              = (255,0,0)
thickness              = 2
lineType               = 2

#habilitar camara
source = 1 # 1 porque usamos otro source (Cámara virtual) y el 0 es la cámara por defecto
cam = cv2.VideoCapture(source)

classes = ["Gallina", "Gato", "Perro"]

umbral  = 0.9
new_dim = (50,50)
continuar = True
#realizar un proceso de forma indefinida
while continuar: 
#for i in range(10):
    #
    retval, frame = cam.read()
    #gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)    
        
    img_noisy_removed = remove_noise(frame)
    img_normalized = normalize_size(img_noisy_removed)
    img_contrast_adjusted = adjust_contrast(img_normalized)

    features = extract_hog_features(img_contrast_adjusted)
    pcaImg = pca.transform([features])

    ans = clf.predict(pcaImg)
    
    font = cv2.FONT_HERSHEY_SIMPLEX 
    org = (50, 50) 
    fontScale = 1
    color = (255, 0, 0) 
    thickness = 2
    image = cv2.putText(frame, classes[ans[0]], org, font, fontScale, color, thickness, cv2.LINE_AA)

    cv2.imshow('src',  frame)
    cv2.imshow('filters', cv2.resize(img_contrast_adjusted, (600, 600)))
    cv2.imshow('feats', cv2.resize(extract_hog_features2(img_contrast_adjusted), (600, 600)))
    
    
    key =  cv2.waitKey(40)
    if key == 27 :
            break
    
cam.release()
cv2.destroyAllWindows()
		
print('\nDone')


Done
